# This file has the sole purpose of conducting tests for the negotiation model between pedestrians.

### A first objective is to test and develop a negotiation model based on social choice theory, regardless of the motion part

The conducted tests will mainly use Mesa, a Python library developed to work with agent-based models.

# Social model

## Utility functions

In [486]:
import mesa

# Data visualization tools.
import seaborn as sns

# Has multi-dimensional arrays and matrices. Has a large collection of
# mathematical functions to operate on these arrays.
import numpy as np

# Data manipulation and analysis.
import pandas as pd

import copy
# Path finding
# @ https://www.youtube.com/watch?v=8SigT_jhz4I
from pathfinding.core.grid import Grid
from pathfinding.finder.a_star import AStarFinder
from pathfinding.core.diagonal_movement import DiagonalMovement

In [724]:
map1 = [[0,0,2,2,0],
        [1,0,0,0,1],
        [0,0,0,0,0],
        [2,2,2,2,0],
        [0,0,3,0,0]]

map2 = [[0,0,0,2,0,0,1],
        [0,0,0,0,2,2,0],
        [0,0,2,0,0,0,0],
        [0,0,0,0,0,0,0],
        [0,0,2,0,2,2,0],
        [0,0,2,0,2,0,0],
        [3,0,2,2,0,0,1]    
       ]

map3 =  [[0, 0, 0, 0, 2, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 2, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 2, 2, 0, 0, 0, 1],
         [0, 0, 0, 0, 2, 0, 0, 0, 0, 0],
         [0, 0, 0, 2, 2, 0, 0, 0, 0, 0],
         [3, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 2, 0, 2, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 2, 0, 0, 0, 1],
         [0, 0, 0, 0, 0, 2, 2, 0, 0, 0],
         [0, 0, 0, 0, 2, 0, 0, 0, 0, 0],
        ]

map4 = [
 [0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 2, 0, 0, 0, 0],
 [0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 2, 0, 0, 1, 0],
 [0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0],
 [0, 3, 0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 1, 0],
 [0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 2, 0, 0, 0, 0],
 [0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 2, 0, 0, 0, 0],
 [0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 2, 0, 0, 0, 0],
]

maps = [map1, map2, map3, map4]

def transform_grid(map):
    """
        Short utility function to only consider free (1) and obstacles (0) in the map.
        Used to collaborate with the pathfinding library.
    """
    new_map = []
    for row in map:
        new_col = []
        for col in row:
            if col == 2: # Obstacle
                new_col.append(0)
            else:
                new_col.append(1)
        new_map.append(new_col)
    return new_map

print(transform_grid(map1))

def convert_coords(pos):
    """
        Convert real coordinates to integer coordinates.
    """
    return (round(pos[0]), round(pos[1]))

def collides(path1, path2):
    if len(path2) < len(path1):
        path1, path2 = path2, path1
    for i in range(1, len(path1)):
        p1 = path1[i]
        p2 = path2[i]
        if p1.x == p2.x and p1.y == p2.y:
            return (p1.x, p1.y)
    return None

[[1, 1, 0, 0, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [0, 0, 0, 0, 1], [1, 1, 1, 1, 1]]


In [3]:
mapp = transform_grid(map1[::-1])
for r in mapp:
    print(r)
grid = Grid(height=5, width=5, matrix = mapp, inverse=False, grid_id=1)

grid_self_pos = (0, 1)

start = grid.node(grid_self_pos[0], grid_self_pos[1])
end = grid.node(2,4)

finder = AStarFinder(diagonal_movement = DiagonalMovement.always)
path, runs = finder.find_path(start, end, grid)

print("Found path:")
for p in path:
    print((p.x, p.y), end='; ')
print()

[1, 1, 1, 1, 1]
[0, 0, 0, 0, 1]
[1, 1, 1, 1, 1]
[1, 1, 1, 1, 1]
[1, 1, 0, 0, 1]
Found path:



### A first jet on preferences

In [4]:
# Give two virtual agents their features.
motivations = [np.random.rand(), np.random.rand()]
goals = [(5,5), (-2, -3)]
positions = [(0, 0), (3,4)]
trajectories = [5, 4] # paths sizes
speed = [2.4, 1.8] # m/s

# Let the first proposed cost function be defined, given K1=K2=K3=1
def cost(agent):
    mot = motivations[agent]
    pos = np.array(positions[agent])
    goal = np.array(goals[agent])
    dist = np.linalg.norm(pos-goal)
    size = trajectories[agent]
    print(f"dist of agent {agent}: {dist}")
    return round(size / dist + speed[agent] + (1 / mot) + np.random.uniform(0, .1), 3)

# Same function adapted for the model below.
def path_cost(agent, path):
    mot = agent.motivation
    pos = agent.position
    goal = agent.objective
    speed = agent.speed
    dist = agent.model.grid.get_distance(pos, goal)
    size = len(path)
    return round(size / dist + speed + (1 / mot) + np.random.uniform(0, .1), 3)

for agent in range(2):
    print(f"cost of agent {agent}: {cost(agent)}")

dist of agent 0: 7.0710678118654755
cost of agent 0: 16.546
dist of agent 1: 8.602325267042627
cost of agent 1: 4.524


## Agent model

In [693]:
class Ped(mesa.Agent):
    
    def __init__(self, model, pos):
        super().__init__(model)

        # If the agent still didn't reach its objective
        self.active = True

        # Traits
        self.motivation = min(.3, np.random.rand()) # 0-1
        self.speed = np.random.rand() + 1
        
        self.position = pos
        self.direction = None
        self.alert = False
        self.wait = False
        self.path = None
        self.negotiating = False
        self.current_cost = -1
    
    def set_objective(self, pos):
        self.objective = pos

    def find_path(self, obstacle=None, diag=True):
        #TODO add current positions as obstacles
        grid = None
        if obstacle is None:
            grid = Grid(height=len(self.model.map), width=len(self.model.map[0]), matrix = self.model.map, inverse=False, grid_id=1)
        else:
            matrix = copy.deepcopy(self.model.map)
            x, y = obstacle
            matrix[y][x] = 0
            grid = Grid(height=len(self.model.map), width=len(self.model.map[0]), matrix = matrix, inverse=False, grid_id=2)
        
        grid_self_pos = convert_coords(self.position)

        start = grid.node(grid_self_pos[0], grid_self_pos[1])
        end = grid.node(self.objective[0], self.objective[1])

        finder = AStarFinder(diagonal_movement = DiagonalMovement.always if diag else DiagonalMovement.never)
        path, runs = finder.find_path(start, end, grid)

        return path

    def get_direction(self):
        if self.path and len(self.path) > 1:
            end = np.array((self.path[1].x, self.path[1].y))
            start = np.array((self.position[0], self.position[1]))
            data = end - start

            data = data / np.linalg.norm(data)
            
            def round_to_half_unit(n):
                return round(n * 2) / 2
            
            self.direction = list(map(round, data))
    
    def negotiate(self, neigh):
        self.negotiating = True
        self.colliding_point = collides(self.path, neigh.path)
        if self.colliding_point is not None:
            self.alert = True
            self.bargainer = neigh
            match self.model.params["negotiation"]:
                case "wait_only":
                    if path_cost(self, self.path) < path_cost(self.bargainer, self.bargainer.path):
                        self.wait = True
                    self.bargainer.current_cost = path_cost(self.bargainer, self.bargainer.path)
                    self.current_cost = path_cost(self, self.path)
                case "alt_or_wait":
                    alt_path = self.find_path(obstacle = self.colliding_point, diag = False)
                    alt_pathneigh = self.bargainer.find_path(obstacle = self.colliding_point, diag = False)
                    if alt_path == [] or alt_pathneigh == []:
                        if path_cost(self, self.path) > path_cost(self.bargainer, self.bargainer.path):
                            self.wait = True
                        else:
                            self.bargainer.wait = True
                    elif path_cost(self, alt_path) > path_cost(self.bargainer, alt_pathneigh):
                        self.bargainer.path = alt_pathneigh
                        self.bargainer.get_direction()
                    else:
                        self.path = alt_path
                        self.get_direction() # update direction!!
                    #TODO à mettre à jour
                    self.current_cost = path_cost(self, alt_path)
                    self.bargainer.current_cost = path_cost(self.bargainer, alt_pathneigh)

    def init(self):
        # First cognition step, to show trajectories.
        self.path = self.find_path()
        self.get_direction()

    def step(self):
        if self.position == self.objective:
            self.active = False
            return
        # Reset (in order)
        self.negotiating = False
        self.alert = False

        # Cognition is supposed to be before motion, in order, however the initialisation step takes care of it.
        
        ### Move agent
        
        #self.position = (self.position[0] + .2, self.position[1] + .2)
        if not self.wait:
            if self.direction is not None:
                self.position = (self.position[0] + self.direction[0], self.position[1] + self.direction[1])
                self.model.grid.move_agent(self, self.position)
        
        ### Cognition
        
        self.path = self.find_path()
        self.get_direction()

        # Reset
        self.wait = False

In [101]:
class Obstacle(mesa.Agent):
    
    def __init__(self, model, pos):
        super().__init__(model)
        self.position = pos

    def step(self):
        pass

class Objective(mesa.Agent):
    
    def __init__(self, model, pos):
        super().__init__(model)
        self.position = pos

    def step(self):
        pass

## Global model

### Negotiation process

In [737]:
class Vote():
    def __init__(self, a1, a2):
        self.neg_schedule = mesa.time.RandomActivation(self)
        self.neg_schedule.add(a1)
        self.neg_schedule.add(a2)
        self.a1 = a1
        self.a2 = a2

    def start(self):
        n = 0
        t1 = a1.path
        while n < Gamma:
            negotiator = self.neg_schedule.agents[0] # Take a random agent
            # Negotiate with the other
            if a == self.a1:
                a.negotiate(self.a2)
            else:
                a.negotiate(self.a1)
            neg_step += 1

In [ ]:
class Negotiation():
    def __init__(self, a1, a2):
        self.a1 = a1
        self.a2 = a2

    def start(self):
        

In [739]:
class SpaceModel(mesa.Model):

    def __invert_y_coord(self, map):
        return map[::-1]

    def __init__(self, params, seed=None):
        """
            Create a continuous space model with discretized time with the given map (2D array).
        """
        super().__init__()

        # Retrieve params
        self.params = params
        map = params["map"]

        # Set seed
        if seed is not None:
            np.random.seed(seed)
            
        # Params
        self.EXTRA_BORDER = 0
        self.map = self.__invert_y_coord(map)
        self.grid = mesa.space.ContinuousSpace(x_max = len(map[0]) + self.EXTRA_BORDER, y_max = len(map) + self.EXTRA_BORDER, torus = True, x_min=-self.EXTRA_BORDER, y_min=-self.EXTRA_BORDER)
        self.schedule = mesa.time.RandomActivation(self)
        self.obstacles = []
        
        self.num_agents = 0
        # Create agents
        for y in range(len(map)):
            for x in range(len(map[y])):
                object = self.map[y][x]
                if object == 1: # Agent
                    a = Ped(self, (x, y))
                    a.unique_id = self.num_agents
                    self.num_agents += 1
                    self.schedule.add(a)
                    self.grid.place_agent(a, a.position)
                elif object == 2: # Obstacle
                    o = Obstacle(self, (x, y))
                    self.grid.place_agent(o, o.position)
                    self.obstacles.append(o.position)
                elif object == 3: # Common objective, for now
                    o = Objective(self, (x, y))
                    self.objective = o
                    self.grid.place_agent(o, o.position)
                # Otherwise empty
        
        # Update map for pathfinding
        self.map = transform_grid(self.map)
        
        # Add objective and init agents
        for agent in self.schedule.agents:
            if isinstance(agent, Ped):
                agent.set_objective(self.objective.position)
                agent.init()

    def step(self):
        DELTA = 2 # attention at 100% here
        # Make step all agents
        self.schedule.step()
        
        for a in self.schedule.agents:
            if not a.active:
                continue

            ### Negotiation
            # Negotiation has to happen in all agent's loop after they *all* moved first in order to deal with updated positions. 
            
            # self.position is discrete for now, however, we should consider comparing distances in float later
            for neigh in self.grid.get_neighbors(a.position, DELTA, False):
                if isinstance(neigh, Ped) and neigh.active and not neigh.alert:
                    if neigh.unique_id != a.unique_id:
                        a.negotiate(neigh)
                        break
            
            ### Debug
            if a.pos != a.objective:
                print(f"agent {a.unique_id}'s cost: {path_cost(a, a.path)}")

## Running

In [505]:
params = {
    "map": map2,
    "negotiation": "wait"
}
mod = SpaceModel(params)

In [506]:
mod.step()

agent 1's cost: 8.293
agent 0's cost: 8.804


## Old visualisation

In [743]:
from mesa.visualization import SolaraViz, make_plot_measure, make_space_matplotlib
from mesa.experimental import JupyterViz

import matplotlib.pyplot as plt

def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

cmap = plt.colormaps.get_cmap('hsv')

def agent_portrayal(agent):
    if isinstance(agent, Ped):
        color = cmap(agent.unique_id*10)
        if agent.alert:
            color = cmap(agent.unique_id*10 - 10)
        return {
            "color": color,
            "size": 50,
        }
    elif isinstance(agent, Obstacle):
        return {
            "color": "tab:red",
            "shape": "s",
            "size":100,
        }
    elif isinstance(agent, Objective):
        return {
            "color": "tab:green",
            "shape": "x",
            "size":80,
        }

model_params = {
    "map": map2,
    "negotiation": "wait_only"
}

# Create initial model instance
model1 = SpaceModel(model_params)

SpaceGraph = make_space_matplotlib(agent_portrayal)

page = SolaraViz(
    model1,
    components=[SpaceGraph],
    model_params=model_params,
    name="Space Model",
)
# This is required to render the visualization in the Jupyter notebook
page

# injecter des exemples
# approfondir choix social et voir après les propriétés et thms du choix social, évaluer le modèle sur des exemples spéciqiues concrets qui marchent
# écrire concis et ajouter formalisme

Cannot show ipywidgets in text

# Visualization

In [ ]:
# 0.11234434484576172
# 0.5729196454356302
# 0.5311117517269073

In [744]:
from matplotlib.figure import Figure
import matplotlib as mpl
import solara

# Components

@solara.component
def MarkdownWithColor(txts, color):
    for txt in txts:
        solara.Markdown(txt, style={"color": color})

def normalize_to_8bit(rgb_normalized):
    # Multiply each component by 255 and round to the nearest integer
    return tuple(int(component * 255) for component in rgb_normalized)

import webcolors

# Function to find the closest color name
def closest_color(requested_color):
    min_colors = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_color[0]) ** 2
        gd = (g_c - requested_color[1]) ** 2
        bd = (b_c - requested_color[2]) ** 2
        min_colors[(rd + gd + bd)] = name
    return min_colors[min(min_colors.keys())]

def get_contrasted_color(rgb):
    # Calculate brightness using the luminosity method
    brightness = 0.299 * rgb[0] + 0.587 * rgb[1] + 0.114 * rgb[2]
    
    # Return black for light colors, white for dark colors
    if brightness > 128:
        return (0, 0, 0)  # Black
    else:
        return (255, 255, 255)  # White

def get_complementary_color(rgb):
    # Calculate the complementary color
    return tuple(255 - component for component in rgb)

# Main visualization

step = solara.reactive(0)
seed = solara.reactive(np.random.randint(0, 2<<32-1))

@solara.component
def SpaceVisualization(clazz, params):
    
    ### Utility
    def format_path(path):
        if path is None:
            return "[]"
        cell = lambda c : (c.x, c.y)
        return list(map(cell, path))
    
    ### Model
    model, set_model = solara.use_state(clazz(params, seed=int(seed.value)))

    fig = Figure(figsize=(6, 6))
    ax = fig.subplots()
    #ax.grid(which='major', color='black', linestyle='--', linewidth=.2)
    grid = model.grid

    # Plot borders (temp fix for fixed grid)
    c = .5
    for (x, y) in [(-c, -c), (grid.width+c, -c), (grid.width+c, grid.height+c), (-c, grid.height+c)]:
        ax.scatter(x, y, marker='+', c='blue')
    
    # Plot extended borders
    border = 0
    for (x, y) in [(-border, -border), (grid.width+border, -border), (grid.width+border, grid.height+border), (-border, grid.height+border)]:
        ax.scatter(x, y, marker='+', c='black', linewidths=0)

    # Plot agents
    cmap = mpl.colormaps['plasma']

    # -- Take colors at regular intervals spanning the colormap.
    colors = cmap(np.linspace(0, 1, len(model.schedule.agents)))
    
    for a in model.agents:
        if not isinstance(a, Ped):
            continue ## We may skip this by removing other non-agents, however they may remain useful to consider neighbor obstacles.
        
        x, y = a.position
        # -- Trajectory
        if a.active and a.direction is not None and len(a.path) > 1:
            xs, ys = zip(*format_path(a.path[:min(3, len(a.path))]))
            c = colors[a.unique_id]
            # Add continuous arrow at the end of the last segment
            ax.annotate('', xy=(xs[-1], ys[-1]), xytext=(xs[-2], ys[-2]), arrowprops=dict(edgecolor=c, facecolor=c, linewidth=1.2, width=1.2, headwidth=7, headlength=4.5, alpha=.25))
            #ax.quiver(xs[-2], ys[-2], xs[-1] - xs[-2], ys[-1] - ys[-2], angles='xy', scale_units='xy', scale=1, color='b', linewidth=2)
            # Plot segment just after
            ax.plot(xs, ys, linestyle=':', color=c, linewidth=2.8, zorder=1, alpha=1)

        # -- Speed
        ax.arrow(x, y, a.direction[0] * a.speed, a.direction[1] * a.speed, width = 0.03, color='green', alpha=.5)
        # -- Agent
        ax.scatter(x, y, linewidths=12, alpha=1, color=colors[a.unique_id])
        # -- Id
        ax.annotate(a.unique_id, (x+.1, y+.1), c='black')
        # -- Virtual obstacle
        if a.alert:
            x, y = a.colliding_point 
            ax.scatter(x, y, linewidths=18, alpha=.5, c='purple')
    
    # Plot obstacles
    for (x, y) in model.obstacles:
        #ax.fill([x-.5, x-.5, x+.5, x+.5], [y-.5, y+.5, y+.5, y-.5], c='red')
        ax.scatter(x, y, c='red', linewidths=8, marker='s')
        
    # Plot objective (to change)
    x, y = model.objective.position
    ax.scatter(x, y, c='green', marker='x', linewidths=10)
    
    # Plot grid
    # Manually set major and minor ticks to ensure spacing is correct
    ax.set_xticks(np.arange(-border-2, grid.width+border+2, 1), minor=False)
    ax.set_yticks(np.arange(-border-2, grid.height+border+2, 1), minor=False)
    ax.set_xticks(np.arange(-border-2, grid.width+border+2, .5), minor=True)
    ax.set_yticks(np.arange(-border-2, grid.height+border+2, .5), minor=True)
    ax.grid(which="minor", color="gray", linestyle='-', linewidth=.5)
    ax.set_axisbelow(True)
    fig.tight_layout()
    
    ### Model UI
    def model_update():
        model.step()
        step.value += 1
        # Plot agents
        for a in model.agents:
            ax.scatter(a.position[0], a.position[1], linewidths=10)

    def reset_model():
        step.value = 0
        set_model(clazz(params, seed=int(seed.value)))

    def set_random_seed():
        rng = np.random.default_rng()
        seed.set(rng.integers(0, 2<<32-1))
        reset_model()

    def set_map(map):
        params["map"] = maps[int(map.split(" ")[-1])-1]
        reset_model()
    
    with solara.VBox() as main:
        with solara.Columns():
            # Main
            solara.FigureMatplotlib(fig, dependencies=[ax])
            for a in model.agents:
                if not isinstance(a, Ped):
                    continue
                col = normalize_to_8bit(tuple(colors[a.unique_id][:3]))
                title = f"""<h1 style="text-shadow: 1px 1px 1px black;color: {closest_color(col)};">Agent {a.unique_id}:</h1>"""
                txt = f"""<h3 style="line-height:1.5;background-color: rgba(0, 100, 120, 0.5); filter: blur(0.25px); color:white">
                <ul>
                <li> Real position: {a.position} | Cell position: {convert_coords(a.position)} </li>
                <li> Direction: {a.direction} </li>
                <li> Speed: {round(a.speed, 3)} </li>
                <li> Objective: {a.objective} </li>
                <li> Path: {format_path(a.path)} </li>
                <li> Negotiating: {a.negotiating} </li>
                <li> Alert: {a.alert} </li>
                <li> Envisioned cost: {a.current_cost} </li>
                </h3>
                """
                MarkdownWithColor([title, txt], "black")
        solara.Text(f"Negotiating mode: {params['negotiation']}")
                
        # UI
        solara.Button(label=f"Step {step}", on_click=model_update)
        solara.Button(label=f"Reset (seed: {seed.value})", on_click=reset_model)
        with solara.Row():
            solara.Button("Random", on_click=set_random_seed)
            solara.InputText("Set Seed", value=seed)
        with solara.Columns():
            solara.Select(label="Map",
                          values=list(map(lambda x: f"Map {x}", list(range(1, len(maps)+1)))),
                          value="Map 2",
                          on_value=set_map)
        
    return main

params = {
    "map": map2,
    "negotiation": "alt_or_wait" # wait_only
}

SpaceVisualization(SpaceModel, params)

Cannot show ipywidgets in text

In [ ]:
## Waiting
# 1946990246 : blue goes (dominant case)
# 2312328177 : yellow goes
## Alternative 1-comparison
# 2356443858 : blue goes (dominant case)
# 1182916481 : yellow goes
## 3 agents
# 2340644788 : no-collision